<a href="https://colab.research.google.com/github/frankwillard/NBA-MVP-Model/blob/main/MVP_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
from math import exp
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, get_scorer, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, mutual_info_regression, RFE, SelectFromModel, SequentialFeatureSelector
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [114]:
# Read in cleaned player data
model_df = pd.read_csv("https://raw.githubusercontent.com/frankwillard/NBA-MVP-Model/main/Seasons_Stats.csv", index_col=0)
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24691 entries, 0 to 24690
Data columns (total 52 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    24624 non-null  float64
 1   Player  24624 non-null  object 
 2   Pos     24624 non-null  object 
 3   Age     24616 non-null  float64
 4   Tm      24624 non-null  object 
 5   G       24624 non-null  float64
 6   GS      18233 non-null  float64
 7   MP      24138 non-null  float64
 8   PER     24101 non-null  float64
 9   TS%     24538 non-null  float64
 10  3PAr    18839 non-null  float64
 11  FTr     24525 non-null  float64
 12  ORB%    20792 non-null  float64
 13  DRB%    20792 non-null  float64
 14  TRB%    21571 non-null  float64
 15  AST%    22555 non-null  float64
 16  STL%    20792 non-null  float64
 17  BLK%    20792 non-null  float64
 18  TOV%    19582 non-null  float64
 19  USG%    19640 non-null  float64
 20  blanl   0 non-null      float64
 21  OWS     24585 non-null  float64
 22

In [115]:
MVP_dict = {
    2023: 'Joel Embiid',
    2022: 'Nikola Jokic',
    2021: 'Nikola Jokic',
    2020: 'Giannis Antetokounmpo',
    2019: 'Giannis Antetokounmpo',
    2018: 'James Harden',
    2017: 'Russell Westbrook',
    2016: 'Stephen Curry',
    2015: 'Stephen Curry',
    2014: 'Kevin Durant',
    2013: 'LeBron James',
    2012: 'LeBron James',
    2011: 'Derrick Rose',
    2010: 'LeBron James',
    2009: 'LeBron James',
    2008: 'Kobe Bryant',
    2007: 'Dirk Nowitzki',
    2006: 'Steve Nash',
    2005: 'Steve Nash',
    2004: 'Kevin Garnett',
    2003: 'Tim Duncan',
    2002: 'Tim Duncan',
    2001: 'Allen Iverson',
    2000: "Shaquille O'Neal",
    1999: 'Karl Malone',
    1998: 'Michael Jordan',
    1997: 'Karl Malone',
    1996: 'Michael Jordan',
    1995: 'David Robinson',
    1994: 'Hakeem Olajuwon',
    1993: 'Charles Barkley',
    1992: 'Michael Jordan',
    1991: 'Michael Jordan',
    1990: 'Magic Johnson',
    1989: 'Magic Johnson',
    1988: 'Michael Jordan',
    1987: 'Magic Johnson',
    1986: 'Larry Bird',
    1985: 'Larry Bird',
    1984: 'Larry Bird',
    1983: 'Moses Malone',
    1982: 'Moses Malone',
    1981: 'Julius Erving',
    1980: 'Kareem Abdul-Jabbar',
    1979: 'Moses Malone',
    1978: 'Bill Walton',
    1977: 'Kareem Abdul-Jabbar',
    1976: 'Kareem Abdul-Jabbar',
    1975: 'Bob McAdoo',
    1974: 'Kareem Abdul-Jabbar',
    1973: 'David Cowens',
    1972: 'Kareem Abdul-Jabbar',
    1971: 'Kareem Abdul-Jabbar',
    1970: 'Willis Reed',
    1969: 'Westley Unseld',
    1968: 'Wilt Chamberlain',
    1967: 'Wilt Chamberlain',
    1966: 'Wilt Chamberlain',
    1965: 'Bill Russell',
    1964: 'Oscar Robertson',
    1963: 'Bill Russell',
    1962: 'Bill Russell',
    1961: 'Bill Russell',
    1960: 'Wilt Chamberlain',
    1959: 'Bob Pettit',
    1958: 'Bill Russell',
    1957: 'Bob Cousy',
    1956: 'Bob Pettit'
}

In [116]:
model_df = model_df[model_df['Year'].notna()]

In [117]:
model_df['Player'] = model_df['Player'].astype(str)
model_df['Year'] = model_df['Year'].astype(int)

<ipython-input-117-1b5862c504dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Player'] = model_df['Player'].astype(str)
<ipython-input-117-1b5862c504dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Year'] = model_df['Year'].astype(int)


In [118]:
model_df['Player'] = model_df['Player'].str.replace('*', '')

<ipython-input-118-24ad200c87b6>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  model_df['Player'] = model_df['Player'].str.replace('*', '')
<ipython-input-118-24ad200c87b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Player'] = model_df['Player'].str.replace('*', '')


In [119]:
model_df['MVP'] = model_df.apply(lambda row: int(row['Player'] in MVP_dict.values() and MVP_dict.get(row['Year']) == row['Player']), axis=1)

## Feature Engineering

In [120]:
model_df = model_df[model_df['G'] >= model_df.groupby('Year')['G'].transform(lambda x: x.max() / 2)]

In [121]:
# min_year = model_df[model_df['MVP'] == 1]['Year'].min()
min_year = model_df[model_df['VORP'].notna()]['Year'].min()

In [122]:
model_df = model_df[model_df['Year'] >= min_year]

In [123]:
model_df.loc[:, 'PPG'] = model_df['PTS'] / model_df['G']
model_df.loc[:, 'WSPG'] = model_df['WS'] / model_df['G']
model_df.loc[:, 'APG'] = model_df['AST'] / model_df['G']
model_df.loc[:, 'TRPG'] = model_df['TRB'] / model_df['G']
model_df.loc[:, 'VORPPG'] = model_df['VORP'] / model_df['G']

## Test Data Split

In [229]:
# Need to get team seed data

model_cols = ['PPG', 'WSPG', 'TRPG', 'APG', 'BPM', 'TS%']
model_df = model_df[model_df[model_cols].notna().all(axis=1)]

In [230]:
train_data = model_df[model_df['Year'] < 2015]
test_data = model_df[model_df['Year'] >= 2015]

In [231]:
X_train = train_data[model_cols]
y_train = train_data[['MVP']]

X_test = test_data[model_cols]
y_test = test_data[['MVP']]

## Modeling

In [232]:
classifier = LogisticRegression(random_state = 1)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

### Model Coefficients

In [233]:
for col, coef in zip(model_cols, classifier.coef_[0]):
  print(f"{col}: {exp(coef)}")

PPG: 1.3060261567184852
WSPG: 1.7538246364030368
TRPG: 1.4373323218828467
APG: 1.5419142536803672
BPM: 1.3604883281876037
TS%: 1.2305281355525854


## Train Prediction

In [234]:
train_predictions = classifier.predict_proba(X_train)
train_data['prob'] = train_predictions[:,1]

<ipython-input-234-b0c92ec72889>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['prob'] = train_predictions[:,1]


## Correct Train Predictions

In [235]:
top1_train_data = train_data.groupby('Year')['prob'].nlargest(1).reset_index(level=0, drop=True)
mvp_data = train_data[train_data['MVP'] == 1]
correct_train_data = train_data[(train_data['prob'].isin(top1_train_data)) & (train_data['MVP'] == 1)]

In [236]:
correct_train_data[['Year', 'Player', 'MVP', 'prob']].sort_values(by=['prob'], ascending=False)

,Year,Player,MVP,prob
4375,1976,Kareem Abdul-Jabbar,1,0.888153
20207,2010,LeBron James,1,0.800130
8869,1988,Michael Jordan,1,0.711502
14806,2000,Shaquille O'Neal,1,0.677066
19611,2009,LeBron James,1,0.675737
3852,1974,Kareem Abdul-Jabbar,1,0.601981
7574,1985,Larry Bird,1,0.547597
4650,1977,Kareem Abdul-Jabbar,1,0.519370
21933,2013,LeBron James,1,0.510270
16674,2004,Kevin Garnett,1,0.486517


## Incorrect Train Predictions

In [237]:
incorrect_train_data = train_data[(train_data['prob'].isin(top1_train_data)) & (train_data['MVP'] == 0)]

In [238]:
incorrect_train_data[['Year', 'Player', 'MVP', 'prob']].sort_values(by=['prob'], ascending=False)

,Year,Player,MVP,prob
9297,1989,Michael Jordan,0,0.900994
9747,1990,Michael Jordan,0,0.671288
19017,2008,LeBron James,0,0.658296
4098,1975,Kareem Abdul-Jabbar,0,0.630355
11731,1994,David Robinson,0,0.622236
8325,1987,Larry Bird,0,0.552585
17918,2006,LeBron James,0,0.467873
11106,1993,Michael Jordan,0,0.415310
16286,2003,Tracy McGrady,0,0.381463
17266,2005,Kevin Garnett,0,0.373765


In [239]:
incorrect_train_data_true_mvp = train_data[(~train_data['prob'].isin(top1_train_data)) & (train_data['MVP'] == 1)]

In [240]:
merged_data = pd.merge(incorrect_train_data, incorrect_train_data_true_mvp, on='Year', suffixes=('1', '2'))

In [241]:
merged_data['disparity'] = merged_data['prob1'] - merged_data['prob2']

In [242]:
merged_data[['Year', 'Player1', 'MVP1', 'prob1', 'Player2', 'MVP2', 'prob2', 'disparity']].sort_values(by=['disparity'], ascending=False)

,Year,Player1,MVP1,prob1,Player2,MVP2,prob2,disparity
18,2008,LeBron James,0,0.658296,Kobe Bryant,1,0.049597,0.608699
16,2006,LeBron James,0,0.467873,Steve Nash,1,0.010336,0.457537
9,1994,David Robinson,0,0.622236,Hakeem Olajuwon,1,0.187931,0.434305
15,2005,Kevin Garnett,0,0.373765,Steve Nash,1,0.002585,0.371181
13,2001,Shaquille O'Neal,0,0.346732,Allen Iverson,1,0.025226,0.321506
1,1978,Kareem Abdul-Jabbar,0,0.365658,Bill Walton,1,0.066345,0.299313
7,1990,Michael Jordan,0,0.671288,Magic Johnson,1,0.376484,0.294804
6,1989,Michael Jordan,0,0.900994,Magic Johnson,1,0.626131,0.274863
14,2003,Tracy McGrady,0,0.381463,Tim Duncan,1,0.126876,0.254587
19,2011,LeBron James,0,0.223210,Derrick Rose,1,0.029276,0.193934


## Who would the model expect to be MVP most?

In [243]:
train_data[(train_data['prob'].isin(top1_train_data))]['Player'].value_counts()

Kareem Abdul-Jabbar    7
LeBron James           7
Michael Jordan         6
Larry Bird             5
David Robinson         3
Shaquille O'Neal       3
Moses Malone           2
Karl Malone            2
Kevin Garnett          2
Tim Duncan             1
Tracy McGrady          1
Dwyane Wade            1
Kevin Durant           1
Name: Player, dtype: int64

## Top 3 Predictions per year (or if MVP true and not top 3)

In [244]:
top3_data = train_data.groupby('Year')['prob'].nlargest(3).reset_index(level=0, drop=True)
mvp_data = train_data[train_data['MVP'] == 1]
filtered_data = train_data[(train_data['prob'].isin(top3_data)) | (train_data['MVP'] == 1)]

In [245]:
filtered_data[['Player', 'Year', 'MVP', 'prob']][(filtered_data['Year'] > 2010)].sort_values(by=['Year', 'prob'], ascending=False).T

,22438,22606,22548,21933,21827,21757,21395,21453,21264,20812,20795,20991
Player,Kevin Durant,Kevin Love,LeBron James,LeBron James,Kevin Durant,Kobe Bryant,LeBron James,Kevin Love,Kevin Durant,LeBron James,Dwight Howard,Derrick Rose
Year,2014,2014,2014,2013,2013,2013,2012,2012,2012,2011,2011,2011
MVP,1,0,0,1,0,0,1,0,0,0,0,1
prob,0.393907,0.316059,0.17951,0.51027,0.120015,0.030866,0.368964,0.06083,0.036202,0.22321,0.029527,0.029276


In [246]:
filtered_data[['Player', 'Year', 'MVP', 'prob']][(filtered_data['Year'] > 2005) & (filtered_data['Year'] < 2011)].sort_values(by=['Year', 'prob'], ascending=False).T

,20207,20458,20071,19611,19877,19752,19017,19164,18825,18698,18460,18295,18569,17918,17742,17843,18020
Player,LeBron James,Dwyane Wade,Kevin Durant,LeBron James,Dwyane Wade,Chris Paul,LeBron James,Chris Paul,Kobe Bryant,Dwyane Wade,LeBron James,Kobe Bryant,Dirk Nowitzki,LeBron James,Kobe Bryant,Kevin Garnett,Steve Nash
Year,2010,2010,2010,2009,2009,2009,2008,2008,2008,2007,2007,2007,2007,2006,2006,2006,2006
MVP,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1
prob,0.80013,0.096367,0.040003,0.675737,0.408152,0.396581,0.658296,0.135896,0.049597,0.126596,0.10299,0.073756,0.030413,0.467873,0.158041,0.103294,0.010336


## Test Prediction

In [247]:
test_predictions = classifier.predict_proba(X_test)
test_data['prob'] = test_predictions[:,1]

<ipython-input-247-7d32ee0cd9ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['prob'] = test_predictions[:,1]


In [248]:
test_data[test_data['Year'] == 2015][['Player', 'Year', 'MVP', 'prob']].sort_values(by=['prob'], ascending=False)[:10]

,Player,Year,MVP,prob
23480,Russell Westbrook,2015,0,0.609444
23114,James Harden,2015,0,0.141838
23154,LeBron James,2015,0,0.084078
23000,Stephen Curry,2015,1,0.077204
22990,DeMarcus Cousins,2015,0,0.055875
23312,Chris Paul,2015,0,0.036082
23012,Anthony Davis,2015,0,0.031642
23103,Blake Griffin,2015,0,0.009334
23472,John Wall,2015,0,0.006317
23215,Damian Lillard,2015,0,0.004965


In [249]:
test_data[test_data['Year'] == 2015].sort_values(by=['prob'], ascending=False).iloc[0]

Year                   2015
Player    Russell Westbrook
Pos                      PG
Age                    26.0
Tm                      OKC
G                      67.0
GS                     67.0
MP                   2302.0
PER                    29.1
TS%                   0.536
3PAr                  0.196
FTr                   0.445
ORB%                    5.9
DRB%                   16.7
TRB%                   11.4
AST%                   47.0
STL%                    3.0
BLK%                    0.5
TOV%                   14.3
USG%                   38.4
blanl                   NaN
OWS                     7.5
DWS                     3.2
WS                     10.6
WS/48                 0.222
blank2                  NaN
OBPM                    8.8
DBPM                    2.2
BPM                    11.0
VORP                    7.6
FG                    627.0
FGA                  1471.0
FG%                   0.426
3P                     86.0
3PA                   288.0
3P%                 

In [250]:
test_data[test_data['Year'] == 2016][['Player', 'Year', 'MVP', 'prob']].sort_values(by=['prob'], ascending=False)[:10]

,Player,Year,MVP,prob
23633,Stephen Curry,2016,1,0.498727
24070,Russell Westbrook,2016,0,0.475827
23722,James Harden,2016,0,0.175545
23781,LeBron James,2016,0,0.165954
23654,Kevin Durant,2016,0,0.158472
23623,DeMarcus Cousins,2016,0,0.036510
23938,Chris Paul,2016,0,0.034330
24060,John Wall,2016,0,0.012802
23849,Kyle Lowry,2016,0,0.010480
23842,Damian Lillard,2016,0,0.010160


In [251]:
test_data[test_data['Year'] == 2017][['Player', 'Year', 'MVP', 'prob']].sort_values(by=['prob'], ascending=False)[:10]

,Player,Year,MVP,prob
24654,Russell Westbrook,2017,1,0.991764
24306,James Harden,2017,0,0.868995
24365,LeBron James,2017,0,0.442972
24207,DeMarcus Cousins,2017,0,0.218387
24206,DeMarcus Cousins,2017,0,0.169725
24239,Kevin Durant,2017,0,0.075212
24115,Giannis Antetokounmpo,2017,0,0.056224
24625,Karl-Anthony Towns,2017,0,0.050828
24218,Anthony Davis,2017,0,0.047280
24646,John Wall,2017,0,0.035775


## Confusion matrix

In [252]:
y_train_pred = classifier.predict(X_train)
y_test_pred = classifier.predict(X_test)

In [253]:
# Output confuson matrix for the training set
cm = confusion_matrix(y_train, y_train_pred)
#[00 01]
#[10 11]
print(cm)

[[12158     6]
 [   31    10]]


In [254]:
# Output confuson matrix for the training set
cm = confusion_matrix(y_test, y_test_pred)
#[00 01]
#[10 11]
print(cm)

[[1084    2]
 [   2    1]]


## Ranked Confusion Matrix (CM of top prediction per year)

In [255]:
train_data['ranked_pred'] = (train_data.groupby('Year')['prob'].transform(lambda x: x == x.max())).astype(int)

<ipython-input-255-15ad6572cf2a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['ranked_pred'] = (train_data.groupby('Year')['prob'].transform(lambda x: x == x.max())).astype(int)


In [256]:
# Output confuson matrix for the training set
cm = confusion_matrix(y_train, train_data['ranked_pred'])
#[00 01]
#[10 11]
print(cm)

[[12144    20]
 [   20    21]]


In [257]:
print(classification_report(y_train, train_data['ranked_pred']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12164
           1       0.51      0.51      0.51        41

    accuracy                           1.00     12205
   macro avg       0.76      0.76      0.76     12205
weighted avg       1.00      1.00      1.00     12205



In [258]:
test_data['ranked_pred'] = (test_data.groupby('Year')['prob'].transform(lambda x: x == x.max())).astype(int)

<ipython-input-258-81c94467c07c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['ranked_pred'] = (test_data.groupby('Year')['prob'].transform(lambda x: x == x.max())).astype(int)


In [259]:
# Output confuson matrix for the training set
cm = confusion_matrix(y_test, test_data['ranked_pred'])
#[00 01]
#[10 11]
print(cm)

[[1085    1]
 [   1    2]]


In [260]:
print(classification_report(y_test, test_data['ranked_pred']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1086
           1       0.67      0.67      0.67         3

    accuracy                           1.00      1089
   macro avg       0.83      0.83      0.83      1089
weighted avg       1.00      1.00      1.00      1089

